# Instruções para reproduzir os resultados do modelo final

In [ ]:
# instalar todas as bibliotecas utilizadas
!pip install -r requirements.txt

## 1. Pré-processar o conjunto de dados

Execute o notebook jupyter 'pre_precessing_pipeline.ipynb' que está no diretório 'processing'. Você deve obter como resultado o arquivo 'esic2023_cleaned.csv' armazenado em 'data'

## 2. Vetorizar conjunto de dados limpo

Execute o notebook jupyter 'vectorizer.ipynb' que está no diretório 'processing'. Você deve obter como resultado os arquivos {'X_train_Tfidf.npz', 'X_test_Tfidf.npz', 'y_train.npy', 'y_test.npy'} armazenado em 'vects'

## 3. Treinar o modelo de SVM

Execute as células abaixo para, por fim, treinar o modelo final

In [46]:
import pandas as pd
import numpy as np
from scipy.sparse import load_npz

from sklearn import svm

from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import LabelEncoder

import pickle

import warnings

warnings.filterwarnings("ignore")

### Carregar vetores de treino e teste

In [44]:
X_train_Tfidf = load_npz('vects/X_train_Tfidf.npz')
X_test_Tfidf = load_npz('vects/X_test_Tfidf.npz')

y_train = np.load('vects/y_train.npy', allow_pickle=True)
y_test = np.load('vects/y_test.npy', allow_pickle=True)

### Otimização de hiperparametros

In [45]:
SVM = svm.SVC()

In [46]:
param_grid = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': [1.0, 2.0, 3.0]
}

In [47]:
grid_search = GridSearchCV(SVM, param_grid, cv=10, scoring='accuracy')

grid_search.fit(X_train_Tfidf, y_train)

print("Melhores hiperparâmetros:", grid_search.best_params_)

Melhores hiperparâmetros: {'C': 2.0, 'kernel': 'rbf'}


In [48]:
accuracy = grid_search.score(X_test_Tfidf, y_test)
print("Acurácia no conjunto de teste:", accuracy)

Acurácia no conjunto de teste: 0.5816666666666667


### Avaliação

In [49]:
SVM = svm.SVC(**grid_search.best_params_)

In [50]:
cv_scores = cross_val_score(SVM, X_train_Tfidf, y_train, cv=10).mean()

print("{:.2f}".format(cv_scores*100), '%')

56.55 %


### Teste

In [51]:
SVM.fit(X_train_Tfidf, y_train)

y_pred = SVM.predict(X_test_Tfidf)

test_scores = accuracy_score(y_pred=y_pred, y_true=y_test)

print("{:.2f}".format(test_scores*100), '%')

58.17 %


### Armazenando modelo treinado

In [54]:
with open('models-dump\SVM.pkl', 'wb') as arquivo:
    pickle.dump(SVM, arquivo)

### Carregando modelo treinado

In [2]:
with open('models-dump\SVM.pkl', 'rb') as arquivo:
    SVM = pickle.load(arquivo)

### Testando com conjunto de teste

In [40]:
X_test_Tfidf = load_npz('predictions_vects/X_test_Tfidf.npz')

y_pred = SVM.predict(X_test_Tfidf)


In [48]:
n_list = []
for n in y_pred:
    if n == 0:
        n_list.append('c1')
    if n == 1:
        n_list.append('c234')
    if n == 2:
        n_list.append('c5')
y_pred = np.array(n_list)

In [51]:
test = pd.read_excel('data/ep1_esic2023_clareza_TEST.xlsx')

test['clarity'] = pd.Series(y_pred)

In [54]:
test.head(6)

,resp_text,clarity
0,"Prezado Sr Sérgio Arantes, Em atenção ao se...",c5
1,A PROGEP respondeu ao seu pedido de informaçã...,c234
2,A senhora somente pode adquirir uma animal d...,c5
3,A solicitação está sendo tratada pela área té...,c234
4,A UFRB possuí dois servidores no Cargo de Aud...,c5
5,A Universidade Federal de Goiás (UFG) não pos...,c5


In [53]:
test.to_csv('predictions/predictions.csv')